# Möbius‑Ladder Explorer (v2) — *Parameterized “Breath” Sweep*

This upgrade lets you **sweep φ‑scales and cycle counts** around the Möbius loop:

```
1) x = seed + tiny                    # inject α
2) x = (φ_scale·φ) / x                # expand (fission)
3) x = 1/x / 9
4) x = 1/x
5) x = x / 9                          # Möbius neutraliser
6) x = x - tiny                       # re‑inject α
   ↺  repeat N cycles
```

* Parameters exposed:  
  * `seed_bits` (list) – Fibonacci word bit‑depths  
  * `k_tiny`        – tiny = α⁻¹ / 10ᵏ  
  * `phi_scales` (list) – try φ_scale 0.7…1.3  
  * `cycles`  (list)  – how many loops  
* Outputs a table of **relative error to α** after each cycle; highlights any < 1 % hits.


In [1]:
import mpmath as mp, itertools, pandas as pd, math
mp.mp.dps = 120

ALPHA_INV = mp.mpf('137.035999084')
ALPHA = 1/ALPHA_INV
PHI = (1 + mp.sqrt(5)) / 2

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_fraction(n_bits):
    bits=fib_word_bits(n_bits)
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(bits))


In [2]:
def breath(x, tiny, phi_scale):
    x = phi_scale*PHI / x
    x = 1/x / 9
    x = 1/x
    x = x / 9
    x = x - tiny
    return x

## Sweep over bit‑depth, φ‑scale, cycles

In [3]:
seed_bits_list = [32, 64, 128]
phi_scales   = [0.75, 0.85, 0.95, 1.05, 1.15]
cycles_list  = [1,2,3,4,5]
k_tiny       = 4  # tiny = α⁻¹ / 10^k

rows=[]
for bits in seed_bits_list:
    seed  = fib_fraction(bits)
    tiny  = ALPHA_INV / (10**k_tiny)
    for phi_scale in phi_scales:
        x = seed + tiny
        for n in cycles_list:
            x = breath(x, tiny, phi_scale)
            rel = abs((x-ALPHA)/ALPHA)
            rows.append(dict(bits=bits, phi_scale=phi_scale, cycles=n, x= mp.nstr(x,25), rel_err=float(rel)))
            if rel < 0.01:
                print('Hit <1%  | bits', bits, 'phi_scale', phi_scale, 'cycles', n, 'rel=', rel)

df = pd.DataFrame(rows)
print('Top 10 smallest errors:')
display(df.sort_values('rel_err').head(10))


Top 10 smallest errors:


,bits,phi_scale,cycles,x,rel_err
23,32,1.15,4,0.2777991004392611565890237,37.068477
48,64,1.15,4,0.2777991005150220147240445,37.068477
73,128,1.15,4,0.2777991005150220147557981,37.068477
18,32,1.05,4,0.2779241009392764834378593,37.085607
68,128,1.05,4,0.2779241010151008039423736,37.085607
43,64,1.05,4,0.2779241010151008039105933,37.085607
13,32,0.95,4,0.2780755590789248877401273,37.106362
63,128,0.95,4,0.2780755591548261386571395,37.106362
38,64,0.95,4,0.278075559154826138625327,37.106362
8,32,0.85,4,0.2782628690987319781636367,37.132030
